# Log probabilities
Working with probabilities can have multiple problems. Two of them are
1. Accuracy. Low probabilities can cause problems because of the machine precision.
2. Speed. Multiplication is more expensive than addition.

For example, calculating the softmax function with the standard definition is numerically unstable.

In [1]:
import numpy as np

def softmax(x: np.ndarray) -> np.ndarray:
    return np.exp(x) / np.sum(np.exp(x))

In [2]:
print(softmax(np.array([1000, 1000, 1000])))
print(softmax(np.array([-1000, -1000, -1000])))

[nan nan nan]
[nan nan nan]


/var/folders/_4/083qbdyn655_tc_ntm7zm0180000gn/T/ipykernel_2927/1398042222.py:4: RuntimeWarning: overflow encountered in exp
  return np.exp(x) / np.sum(np.exp(x))
/var/folders/_4/083qbdyn655_tc_ntm7zm0180000gn/T/ipykernel_2927/1398042222.py:4: RuntimeWarning: invalid value encountered in divide
  return np.exp(x) / np.sum(np.exp(x))


Idea: Instead of working in linear-scale we represent probabilities on a logarithmic scale.  Therefore we now work with values in the intervall $(-\infty, 0]$ instead of $[0, 1]$. This have some advantages: 
$$
\log(x\cdot y)=\log(x)+\log(y)
$$
In other words: multiplication operations in linear-scale become additions in log-scale

Another example:
$$
    \frac{\partial}{\partial x}\log(f(x)g(x)) = \frac{\partial}{\partial x}(\log f(x) + \log g(x)) = \frac{\partial}{\partial x}\log f(x) + \frac{\partial}{\partial x}\log g(x)
$$

To take advantage of log probabilities we need to define the LogSumExp function.

## LogSumExp
$$
    \begin{align*}
        LSE\colon\mathbb{R}^n &\to \mathbb{R}\\
        (x_1, \dotsc, x_n) &\mapsto \log\left(\sum_{i=1}^n \exp(x_i)\right)
    \end{align*}
$$

For LSE it holds:
1. $\max\{x_1, \dotsc, x_n\} \leq LSE(x_1, \dotsc, x_n) \leq \max\{x_1, \dotsc, x_n\} + \log(n)$,

2. $\max\{x_1, \dotsc, x_n\} \leq \frac{1}{t} LSE(tx_1, \dotsc, tx_n) \leq \max\{x_1, \dotsc, x_n\} + \frac{\log(n)}{t}$,

3. $\min{x_1, \dotsc, x_n} \geq \frac{-1}{t}LSE(-tx)\geq \min{x_1, \dotsc, x_n}-\frac{\log(n)}{t}$

4. $\operatorname{grad} LSE(x)=softmax(x)$,

5. $\log(x_1 + \dotsc, x_n) = LSE(\log(x_1), \dotsc, \log(x_n))$,

6. $LSE(x_1, \dotsc, x_n) = c + LSE(x_1 - c, \dotsc, x_n - c)$ for $c\in\mathbb{R}$,

<details>
    <summary>Proof.</summary> 
1. Let $m=\max\{x_1, \dotsc, x_n\}$. Then we see that
$$
    \exp(m) \leq \sum_{i=1}^n\exp(x_i)\leq n\exp(m)
$$
and applying $\log$ yields the result. 

2. For $t>0$ we will show the following inequality
$$
    \max\{x_1, \dotsc, x_n\} \leq \frac{1}{t} LSE(tx_1, \dotsc, tx_n) \leq \max\{x_1, \dotsc, x_n\} + \frac{\log(n)}{t}.
$$
In fact, we replace $x_i$ with $tx_i$ with the previous inequality, i.e.
$$
    t\max\{x_1, \dotsc, x_n\} = \max\{tx_1, \dotsc, tx_n\} \leq LSE(tx_1, \dotsc, tx_n) \leq \max\{tx_1, \dotsc, tx_n\} + \log(n) = t\max\{x_1, \dotsc, x_n\} + \log(n).
$$
Dividing by $t$ yields the result. 

3. It holds $\max{-tx_1, \dotsc, -tx_n} = -t\min{x_1, \dotsc, x_n}$. Plugging this into the above inequality and dividing by $-t$ yields
$$
    \min{x_1, \dotsc, x_n} \geq \frac{-1}{t}LSE(-tx)\geq \min{x_1, \dotsc, x_n}-\frac{\log(n)}{t}.
$$

4. This calculation is straight forward:
$$
    \frac{\partial}{\partial x_i} LSE(x) = \frac{\exp(x_i)}{\sum_{j=1}^d\exp(x_j)}.
$$

5. It holds
$$
    \log(x_1 + \dotsc, x_n) = LSE(\log(x_1), \dotsc, \log(x_n)),
$$
i.e. addition in linear-scale becomes LSE in log-scale.

6. We see that
$$
    LSE(x_1, \dotsc, x_n) = \log\left(\sum_{i=1}^n\exp(x_i)\right) = \log\left(\sum_{i=1}^n\exp(x_i-c)\exp(c)\right) = \log\left(\exp(c)\sum_{i=1}^n\exp(x_i-c)\right) = \log(\exp(c)) + \log\left(\sum_{i=1}^n\exp(x_i-c)\right) = c + LSE(x_1 - c, \dotsc, x_n - c).
$$
</details>

Often we choose $c=\max{x_1, \dotsc, x_n}$ because then the largest exponent will be $0$.

In [3]:
def LogSumExp(x: np.ndarray) -> float:
    c = x.max()
    return c + np.log(np.sum(np.exp(x - c)))

The softmax function
$$
\begin{align*}
    \sigma\colon \mathbb{R}^d &\to \left\{x\in\mathbb{R}^d\vert x_i\geq 0, \sum_{i=1}^{d}x_i = 1\right\}\\
    x &\mapsto \left(\frac{e^{x_i}}{\sum_{j=1}^{d}e^{x_j}}\right)_{1\leq i\leq d}
\end{align*}
$$
is used to normalize vectors. But $x_i$ might be very large, which might result in an overflow or underflow. We can improve the precision by using the LogSumExp function. In fact, let 
$$
    p_i = \frac{\exp(x_i)}{\sum_{j=1}^n\exp(x_j)}.
$$
Then it follows that $\exp(x_i)=p_i\sum_{j=1}^n\exp(x_j)$ and hence by applying $\log$
$$
    x_i = \log(p_i) + \log\left(\sum_{j=1}^n\exp(x_j)\right) = \log(p_i) + LSE(x_1, \dotsc, x_n).
$$
This is equivalent to
$$
    \log(p_i) = x_i - LSE(x_1, \dotsc, x_n)
$$
and therefore
$$
    p_i = \exp(x_i - LSE(x_1, \dotsc, x_n)).
$$
This version is numerically much more stable because we don't have to divide and we can calculate $LSE(x)$ efficiently.

In [4]:
def improved_softmax(x: np.array) -> np.array:
    return np.exp(x - LogSumExp(x))

In [5]:
print(improved_softmax(np.array([1000, 1000, 1000])))
print(improved_softmax(np.array([-1000, -1000, -1000])))

[0.33333333 0.33333333 0.33333333]
[0.33333333 0.33333333 0.33333333]
